In [1]:
from tkinter import *
from tkinter import messagebox
import os
import webbrowser

import numpy as np
import pandas as pd


!pip install fuzzywuzzy
!pip install python-Levenshtein
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

  Obtaining dependency information for python-Levenshtein from https://files.pythonhosted.org/packages/27/89/c45dbdbd479453cfc8c4c1271c9f67fd607deaf84880e31c67b682980456/python_Levenshtein-0.23.0-py3-none-any.whl.metadata
  Obtaining dependency information for Levenshtein==0.23.0 from https://files.pythonhosted.org/packages/2f/04/a557197d0ba601f09fdd9ef9174b61d87706a291cd1e1578728eda3f44ab/Levenshtein-0.23.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for rapidfuzz<4.0.0,>=3.1.0 from https://files.pythonhosted.org/packages/fa/a9/0bcd4017323d40cec6fc1639eddae1f90b5e080be0c62eb34670b69aa009/rapidfuzz-3.6.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/101.1 kB ? eta -:--:--
   ------------------------ --------------- 61.4/101.1 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 101.1/101.1 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ----- -------------------

In [2]:
class HyperlinkManager:
    def __init__(self, text):
        self.text = text
        self.text.tag_config("hyper", foreground="blue", underline=1)
        self.text.tag_bind("hyper", "<Enter>", self._enter)
        self.text.tag_bind("hyper", "<Leave>", self._leave)
        self.text.tag_bind("hyper", "<Button-1>", self._click)
        self.reset()

    def reset(self):
        self.links = {}

    def add(self, action):
        tag = "hyper-%d" % len(self.links)
        self.links[tag] = action
        return "hyper", tag

    def _enter(self, event):
        self.text.config(cursor="hand2")

    def _leave(self, event):
        self.text.config(cursor="")

    def _click(self, event):
        for tag in self.text.tag_names(CURRENT):
            if tag[:6] == "hyper-":
                self.links[tag]()
                return

In [3]:
# Importing the dataset
training_dataset = pd.read_csv("C:/Users/ASUS/Downloads/Healthcarechatbotproject/Healthcarechatbotproject/Code and Datasets/Training.csv")
test_dataset = pd.read_csv("C:/Users/ASUS/Downloads/Healthcarechatbotproject/Healthcarechatbotproject/Code and Datasets/Testing.csv")
MedicalDescription_Df = pd.read_excel("C:/Users/ASUS/Downloads/Healthcarechatbotproject/Healthcarechatbotproject/Code and Datasets/Medicine_descript.xlsx")
CompanyName_Df = pd.read_excel("C:/Users/ASUS/Downloads/Healthcarechatbotproject/Healthcarechatbotproject/Code and Datasets/Company_Info.xlsx")
Ratings_Df = pd.read_excel("C:/Users/ASUS/Downloads/Healthcarechatbotproject/Healthcarechatbotproject/Code and Datasets/Ratingsss.xlsx")
MedicalDescription_Df['S.No'] = CompanyName_Df['S.No']

In [4]:
# Slicing and Dicing the dataset to separate features from predictions
X = training_dataset.iloc[:, 0:132].values
Y = training_dataset.iloc[:, -1].values

In [5]:
# Dimensionality Reduction for removing redundancies
dimensionality_reduction = training_dataset.groupby(training_dataset['prognosis']).max()




In [6]:
# Encoding String values to integer constants
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(Y)

In [7]:
# Splitting the dataset into training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [8]:
# Implementing the Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

# Saving the information of columns containing symptoms
cols     = training_dataset.columns
cols     = cols[:-1]

# Checking the Important features
importances = classifier.feature_importances_
indices = np.argsort(importances)[::-1]
features = cols

# Implementing the Visual Tree
from sklearn.tree import _tree

In [9]:
# Method to simulate the working of a Chatbot by extracting and formulating questions
def print_disease(node):
        print("node: ",node)
        node = node[0]
        print("len: ",len(node))
        val  = node.nonzero() 
        print("val: ",val)
        disease = labelencoder.inverse_transform(val[0])
        return disease
def print_output(node):
    node=node[0]
    val=node.nonzero()
    output=labelencoder.inverse_transform(val[0])
    return output

In [10]:
def recurse(node, depth):
            global val,ans
            global tree_,feature_name,symptoms_present
            indent = "  " * depth
            if tree_.feature[node] != _tree.TREE_UNDEFINED:
                name = feature_name[node]
                threshold = tree_.threshold[node]
                print(type(name))
                hyperlink = HyperlinkManager(QuestionDigonosis.objRef.txtDigonosis)
                strData1="https://www.google.com/search?q="+str(name)
                print("first page link: ",strData1)
                def click1():
                    webbrowser.open_new(strData1)
                QuestionDigonosis.objRef.txtDigonosis.insert(INSERT, strData1, hyperlink.add(click1))
                
                yield name + " ?" +"\n"
                ans = ans.lower()
                if ans == 'yes':
                    val = 1
                else:
                    val = 0
                if  val <= threshold:
                    yield from recurse(tree_.children_left[node], depth + 1)
                else:
                    symptoms_present.append(name)
                    yield from recurse(tree_.children_right[node], depth + 1)
            else:
                strData=""
                present_disease = print_disease(tree_.value[node])
                strData="You may have :" +  str(present_disease)
               
                QuestionDigonosis.objRef.txtDigonosis.insert(END,str(strData)+'\n')                  
                
                red_cols = dimensionality_reduction.columns 
                symptoms_given = red_cols[dimensionality_reduction.loc[present_disease].values[0].nonzero()]
                strData="symptoms present:  " + str(list(symptoms_present))
                QuestionDigonosis.objRef.txtDigonosis.insert(END,str(strData)+'\n')
                strData="symptoms given: "  +  str(list(symptoms_given))
                QuestionDigonosis.objRef.txtDigonosis.insert(END,str(strData)+'\n')                  
                confidence_level = (1.0*len(symptoms_present))/len(symptoms_given)
                strData="confidence level is: " + str(confidence_level)
                QuestionDigonosis.objRef.txtDigonosis.insert(END,str(strData)+'\n')
                strData='The model suggests:'
                QuestionDigonosis.objRef.txtDigonosis.insert(END,str(strData)+'\n')                  
                row = doctors[doctors['disease'] == present_disease[0]]
                print(row)
                strData='Consult '+ str(row['name'].values)
                QuestionDigonosis.objRef.txtDigonosis.insert(END,str(strData)+'\n')  
                strData='VISIT:'
                QuestionDigonosis.objRef.txtDigonosis.insert(END,str(strData)+'\n')    
                hyperlink = HyperlinkManager(QuestionDigonosis.objRef.txtDigonosis)
                strData1=str(row['link'].values[0])
                def click1():
                    webbrowser.open_new(str(row['link'].values[0]))
                QuestionDigonosis.objRef.txtDigonosis.insert(INSERT, strData1, hyperlink.add(click1))
                strData="\n"+'OR TRY BELOW LINK IF THE DOCTOR MENTIONED ABOVE IS NOT AVAILABLE:'
                QuestionDigonosis.objRef.txtDigonosis.insert(END,str(strData)+'\n')
                strData2=str(row['link2'].values[0])
                def click2():
                    webbrowser.open_new(str(row['link2'].values[0]))
                QuestionDigonosis.objRef.txtDigonosis.insert(INSERT, strData2, hyperlink.add(click2))

                
                

In [10]:
def tree_to_code(tree, feature_names):
        global tree_,feature_name,symptoms_present
        tree_ = tree.tree_
        print(tree_)
        feature_name = [
            feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
            for i in tree_.feature
        ]
        print("def tree({}):".format(", ".join(feature_names)))
        symptoms_present = []   

In [11]:
def execute_bot():   
    tree_to_code(classifier,cols)

In [15]:
doc_dataset = pd.read_csv("C:/Users/ASUS/Downloads/Healthcarechatbotproject/Healthcarechatbotproject/Code and Datasets/doctors_dataset.csv", names = ['Name', 'Description',
                                                                                                          'link2a'])

diseases = dimensionality_reduction.index
diseases = pd.DataFrame(diseases)


doctors = pd.DataFrame()
doctors['name'] = np.nan
doctors['link'] = np.nan
doctors['link2'] = np.nan
doctors['disease'] = np.nan

doctors['disease'] = diseases['prognosis']


doctors['name'] = doc_dataset['Name']
doctors['link'] = doc_dataset['Description']
doctors['link2'] = doc_dataset['link2a']



#execute_bot()
from tkinter import *
from tkinter import messagebox
import os
import webbrowser

import numpy as np
import pandas as pd

class QuestionDigonosis(Frame):
    objIter=None
    objRef=None
    def __init__(self,master=None):
        master.title("Question")
        master.geometry("1200x1000")
        QuestionDigonosis.objRef=self
        super().__init__(master=master)
        self["bg"]="light blue"
        self.createWidget()
        self.iterObj=None


    def createWidget(self):
        self.lblQuestion=Label(self,text="Question",width=12,bg="bisque")
        self.lblQuestion.grid(row=0,column=0,rowspan=4)
        self.lblDigonosis = Label(self, text="Digonosis",width=12,bg="bisque")
        self.lblDigonosis.grid(row=4, column=0,sticky="n",pady=5)
        # self.varQuestion=StringVar()
        self.txtQuestion = Text(self, width=100,height=4)
        self.txtQuestion.grid(row=0, column=1,rowspan=4,columnspan=20)
        self.varDiagonosis=StringVar()
        self.txtDigonosis =Text(self, width=100,height=14)
        self.txtDigonosis.grid(row=4, column=1,columnspan=20,rowspan=20,pady=5)
        self.btnNo=Button(self,text="No",width=12,bg="bisque", command=self.btnNo_Click)
        self.btnNo.grid(row=25,column=10)
        self.btnYes = Button(self, text="Yes",width=12,bg="bisque", command=self.btnYes_Click)
        self.btnYes.grid(row=25, column=9)
        self.btnClear = Button(self, text="Clear",width=12,bg="bisque", command=self.btnClear_Click)
        self.btnClear.grid(row=27, column=0)
        self.btnStart = Button(self, text="Start",width=12,bg="bisque", command=self.btnStart_Click)
        self.btnStart.grid(row=27, column=1,columnspan=20,sticky="e")
        self.MDB=Text(self,width=105,height=22)
        self.MDB.grid(columnspan=25,row=32,rowspan=34,column=0,padx=5,pady=20)
        self.btnMD=Button(self,text="Analyze medicine",width="25",bg='light green',command=self.btnMDB_Click)
        self.btnMD.place(relx=0.5,rely=1.0,anchor='s')
        
        
        
        
    def btnMDB_Click(self):
        med_entered=self.MDB.get(1.0, "end-1c")
        Drugs=med_entered.split(",")
        self.MDB.delete(0.0,END)
        QuestionDigonosis.objRef.MDB.insert(INSERT, "MEDICINE YOU ENTERED:\n"+med_entered)
        QuestionDigonosis.objRef.MDB.insert(END, "\n")
        QuestionDigonosis.objRef.MDB.insert(END,"_____________________________________________________________________________________________" )
        QuestionDigonosis.objRef.MDB.insert(END, "\n")
        
   
        def DrugDisease(drugName,MedicalDescription_Df):
            def getDrugNameReason(drugName,MedicalDescription_Df):
                foundedDrugName = MedicalDescription_Df[MedicalDescription_Df["Drug_Name"] == drugName]['Reason'].to_list()
                print("FDN=",foundedDrugName)
                return str(foundedDrugName[0])
            
            drug_Reason = getDrugNameReason(drugName,MedicalDescription_Df)
            QuestionDigonosis.objRef.MDB.insert(END, "\n")
            QuestionDigonosis.objRef.MDB.insert(END, "Your medicine is for: ",drug_Reason)
            return drug_Reason

        def getDrugDescription(drugName,MedicalDescription_Df):
            foundedDrugName = MedicalDescription_Df[MedicalDescription_Df["Drug_Name"] == drugName]['Description'].to_list()
            QuestionDigonosis.objRef.MDB.insert(END, "\n")
            QuestionDigonosis.objRef.MDB.insert(END, "the function of the drug which u entered: ",foundedDrugName[0])
            return str(foundedDrugName[0])
        def DrugRating(drugName,MedicalDescription_Df,CompanyName_Df,Ratings_Df):
            def getDrugNameSNo(drugName,MedicalDescription_Df):
                foundedDrugName = MedicalDescription_Df[MedicalDescription_Df["Drug_Name"] == drugName]
                return int(foundedDrugName['S.No'])
            drug_SNo = getDrugNameSNo(drugName,MedicalDescription_Df)

            def getDrugRatingShortForm(drug_SNo,CompanyName_Df):
                foundedDrugRatingShortForm = CompanyName_Df[CompanyName_Df['S.No'] == drug_SNo]
                for i in foundedDrugRatingShortForm['Rating']:
                    return str(i)
            drugShortForm = getDrugRatingShortForm(drug_SNo,CompanyName_Df)

            def getDrugRating(drugShortForm,Ratings_Df):
                foundedRating = Ratings_Df[Ratings_Df['Short-form'] == drugShortForm]['Rating'].to_list()
                return foundedRating[0]
            drugRating=getDrugRating(drugShortForm,Ratings_Df)
            
            QuestionDigonosis.objRef.MDB.insert(END, "\n")
            QuestionDigonosis.objRef.MDB.insert(END, "The drug Serial Number is "+str(drug_SNo))
            

            
            QuestionDigonosis.objRef.MDB.insert(END, "\n")
            QuestionDigonosis.objRef.MDB.insert(END, "The drug short form is "+drugShortForm)
            drug_Rating = getDrugRating(drugShortForm,Ratings_Df)
            return drug_Rating
        data_Name = MedicalDescription_Df["Drug_Name"].tolist()
        for drug_name in Drugs:
            print(drug_name)
            for i in data_Name:
                val=fuzz.ratio(i,drug_name)

                if val>87:
                    QuestionDigonosis.objRef.MDB.insert(END, "\n")
                    QuestionDigonosis.objRef.MDB.insert(END, "Drug Found !")
                    QuestionDigonosis.objRef.MDB.insert(END, "\n")
                    QuestionDigonosis.objRef.MDB.insert(END, "%match= "+str(val))
                    a=DrugDisease(i,MedicalDescription_Df)
                    QuestionDigonosis.objRef.MDB.insert(END, a,)
                    b=getDrugDescription(i,MedicalDescription_Df)
                    QuestionDigonosis.objRef.MDB.insert(END, b,)
                    drug_rating = DrugRating(i,MedicalDescription_Df,CompanyName_Df,Ratings_Df)
                    QuestionDigonosis.objRef.MDB.insert(END, "\n")
                    QuestionDigonosis.objRef.MDB.insert(END, "Note: The Rating is on a scale of 5")
                    
                    QuestionDigonosis.objRef.MDB.insert(END, "\n")
                    QuestionDigonosis.objRef.MDB.insert(END, "The drug rating :",drug_rating)
                    QuestionDigonosis.objRef.MDB.insert(END, drug_rating)
                    QuestionDigonosis.objRef.MDB.insert(END, "\n")
                    QuestionDigonosis.objRef.MDB.insert(END,"________________________________________________" )
                    QuestionDigonosis.objRef.MDB.insert(END, "\n")

        

   
        
        
        
    
    def btnNo_Click(self):
        global val,ans
        ans='no'
        str1=QuestionDigonosis.objIter.__next__()
        self.txtQuestion.delete(0.0,END)
        self.txtDigonosis.delete(0.0,END)
        strh=str1.partition("?")[0]
        h="https://www.google.com/search?q="+strh
        hyperlink = HyperlinkManager(QuestionDigonosis.objRef.txtDigonosis)
        QuestionDigonosis.objRef.txtDigonosis.insert(END,"search here for better understanding of the symptom: \n")
        def clicksearch():
            webbrowser.open_new(str(h))
        QuestionDigonosis.objRef.txtDigonosis.insert(INSERT, h, hyperlink.add(clicksearch))
        self.txtQuestion.insert(END,str1+"\n")
        
    def btnYes_Click(self):
        global val,ans
        ans='yes'
        self.txtDigonosis.delete(0.0,END)
        str1=QuestionDigonosis.objIter.__next__()
        
        
    def btnClear_Click(self):
        self.txtDigonosis.delete(0.0,END)
        self.txtQuestion.delete(0.0,END)
        
        
    def btnStart_Click(self):
        execute_bot()
        self.txtDigonosis.delete(0.0,END)
        self.txtQuestion.delete(0.0,END)
        self.txtDigonosis.insert(END,"Please Click on Yes or No for the Above symptoms in Question"+"\n")
        QuestionDigonosis.objIter=recurse(0,1)
        str1=QuestionDigonosis.objIter.__next__()
        self.txtQuestion.insert(END,str1+"\n")


class MainForm(Frame):
    main_Root = None
    def destroyPackWidget(self, parent):
        for e in parent.pack_slaves():
            e.destroy()
    def __init__(self, master=None):
        MainForm.main_Root = master
        super().__init__(master=master)
        master.geometry("300x250")
        master.title("Account Login")
        self.createWidget()
    def createWidget(self):
        self.lblMsg=Label(self, text="Select Your Choice", bg="light blue", width="300", height="2", font=("Calibri", 13))
        self.lblMsg.pack()
        self.btnLogin=Button(self, text="Login", height="2", width="30", command=self.lblLogin_Click)
        self.btnLogin.pack()
        self.btnRegister=Button(self, text="Register", height="2", width="30", command=self.btnRegister_Click)
        self.btnRegister.pack()
    def lblLogin_Click(self):
        self.destroyPackWidget(MainForm.main_Root)
        frmLogin=Login(MainForm.main_Root)
        frmLogin.pack()
    def btnRegister_Click(self):
        self.destroyPackWidget(MainForm.main_Root)
        frmSignUp = SignUp(MainForm.main_Root)
        frmSignUp.pack()
class Login(Frame):
    main_Root=None
    def destroyPackWidget(self,parent):
        for e in parent.pack_slaves():
            e.destroy()
    def __init__(self, master=None):
        Login.main_Root=master
        super().__init__(master=master)
        master.title("Login")
        master.geometry("300x250")
        self.createWidget()
        
    def createWidget(self):
        self.lblMsg=Label(self, text="Please enter details below to login",bg="light blue")
        self.lblMsg.pack()
        self.username=Label(self, text="Username * ")
        self.username.pack()
        self.username_verify = StringVar()
        self.username_login_entry = Entry(self, textvariable=self.username_verify)
        self.username_login_entry.pack()
        self.password=Label(self, text="Password * ")
        self.password.pack()
        self.password_verify = StringVar()
        self.password_login_entry = Entry(self, textvariable=self.password_verify, show='*')
        self.password_login_entry.pack()
        self.btnLogin=Button(self, text="Login", width=10, height=1, command=self.btnLogin_Click)
        self.btnLogin.pack()
        
        
    def btnLogin_Click(self):
        username1 = self.username_login_entry.get()
        password1 = self.password_login_entry.get()
        list_of_files = os.listdir("C:/Users/ASUS/Downloads/Healthcarechatbotproject/Healthcarechatbotproject/Code and Datasets")
        print("list of files:",list_of_files)
        print("username1: ",username1)
        if username1 in list_of_files:
            file1 = open(username1, "r")
            print("file1: ",file1)
            verify = file1.read().splitlines()
            print("verify: ",verify)
            if password1 in verify:
                messagebox.showinfo("Sucess","Login Sucessful")
                self.destroyPackWidget(Login.main_Root)
                frmQuestion = QuestionDigonosis(Login.main_Root)
                frmQuestion.pack()
            else:
                messagebox.showinfo("Failure", "Login Details are wrong try again")
        else:
            messagebox.showinfo("Failure", "User not found try from another user\n or sign up for new user")
            
class SignUp(Frame):
    main_Root=None
    def destroyPackWidget(self,parent):
        for e in parent.pack_slaves():
            e.destroy()
    def __init__(self, master=None):
        SignUp.main_Root=master
        master.title("Register")
        super().__init__(master=master)
        master.title("Register")
        master.geometry("300x250")
        self.createWidget()
        
    def createWidget(self):
        self.lblMsg=Label(self, text="Please enter details below", bg="light blue")
        self.lblMsg.pack()
        self.username_lable = Label(self, text="Username * ")
        self.username_lable.pack()
        self.username = StringVar()
        self.username_entry = Entry(self, textvariable=self.username)
        self.username_entry.pack()
        
        self.password_lable = Label(self, text="Password * ")
        self.password_lable.pack()
        self.password = StringVar()
        self.password_entry = Entry(self, textvariable=self.password, show='*')
        self.password_entry.pack()
        self.btnRegister=Button(self, text="Register", width=10, height=1, bg="light blue", command=self.register_user)
        self.btnRegister.pack()
        
    def register_user(self):   
        folder_path = "C:/Users/ASUS/Downloads/Healthcarechatbotproject/Healthcarechatbotproject/Code and Datasets"
        file_path = os.path.join(folder_path,self.username_entry.get())
        with open(file_path, 'w') as f:
            f.write('username:')
            f.write(self.username_entry.get())
            f.write('\n'+'password:')
            f.write(self.password_entry.get()) 
        f.close()
        self.destroyPackWidget(SignUp.main_Root)
        self.lblSucess=Label(root, text="Registration Success", fg="green", font=("calibri", 11))
        self.lblSucess.pack()
        self.btnSucess=Button(root, text="Click Here to proceed", command=self.btnSucess_Click)
        self.btnSucess.pack()     
    def btnSucess_Click(self):
        self.destroyPackWidget(SignUp.main_Root)
        frmQuestion = QuestionDigonosis(SignUp.main_Root)
        frmQuestion.pack()


root = Tk()

frmMainForm=MainForm(root)
frmMainForm.pack()
root.mainloop()

def tree(itching, skin_rash, nodal_skin_eruptions, continuous_sneezing, shivering, chills, joint_pain, stomach_pain, acidity, ulcers_on_tongue, muscle_wasting, vomiting, burning_micturition, spotting_ urination, fatigue, weight_gain, anxiety, cold_hands_and_feets, mood_swings, weight_loss, restlessness, lethargy, patches_in_throat, irregular_sugar_level, cough, high_fever, sunken_eyes, breathlessness, sweating, dehydration, indigestion, headache, yellowish_skin, dark_urine, nausea, loss_of_appetite, pain_behind_the_eyes, back_pain, constipation, abdominal_pain, diarrhoea, mild_fever, yellow_urine, yellowing_of_eyes, acute_liver_failure, fluid_overload, swelling_of_stomach, swelled_lymph_nodes, malaise, blurred_and_distorted_vision, phlegm, throat_irritation, redness_of_eyes, sinus_pressure, runny_nose, congestion, chest_pain, weakness_in_limbs, fast_heart_rate, pain_during_bowel_movements, pain_in_anal_region, bloody_stool, irritation_in_anus, neck_pain, dizziness, cramps, bruising, ob

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_19024\256003481.py", line 189, in btnStart_Click
    QuestionDigonosis.objIter=recurse(0,1)
                              ^^^^^^^
NameError: name 'recurse' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_19024\256003481.py", line 160, in btnNo_Click
    str1=QuestionDigonosis.objIter.__next__()
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute '__next__'
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\Lib\tkinter\__init__.py", li